# Heart Attack Prediction - Data Preprocessing
## קידוד משתנים קטגוריאליים

### שלב 1: טעינת הספריות והנתונים

In [1]:
import pandas as pd
import numpy as np

# טעינת הנתונים
df = pd.read_csv('Project Heart attack data.csv')

# הצגת הנתונים
print('מימדי הנתונים:', df.shape)
df.head()

מימדי הנתונים: (298, 11)


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,ca,target
0,62,1,2,130,231,0,1,146,0,3,1
1,52,1,0,108,233,1,1,147,0,3,1
2,53,1,2,130,246,1,0,173,0,3,1
3,67,1,0,160,286,0,0,108,1,3,0
4,65,0,0,150,225,0,0,114,0,3,0


### שלב 2: זיהוי המשתנים הקטגוריאליים

**משתנים שצריך לקודד (One-Hot Encoding):**
- `cp` - סוג כאב בחזה (0,1,2,3) - קטגוריות ללא סדר
- `restecg` - תוצאות ECG במנוחה (0,1,2) - קטגוריות ללא סדר

**משתנים שלא צריך לקודד:**
- `sex`, `fbs`, `exng` - בינאריים (0/1)
- `ca` - מספרי/סדור (כמות כלי דם)
- `age`, `trtbps`, `chol`, `thalachh` - רציפים

In [5]:
# בדיקת הערכים הייחודיים במשתנים הקטגוריאליים
print('ערכים ייחודיים ב-cp:', df['cp'].unique())
print('ערכים ייחודיים ב-restecg:', df['restecg'].unique())

ערכים ייחודיים ב-cp: [2 0 1 3]
ערכים ייחודיים ב-restecg: [1 0 2]


### שלב 3: קידוד One-Hot Encoding

נשתמש ב-`pd.get_dummies()` עם `drop_first=True` למניעת מולטיקולינאריות

In [6]:
# קידוד המשתנים הקטגוריאליים
# dtype=int כדי לקבל 0/1 במקום True/False
df_encoded = pd.get_dummies(df, columns=['cp', 'restecg'], drop_first=True, dtype=int)

# הצגת העמודות החדשות
print('עמודות לפני הקידוד:', list(df.columns))
print('\nעמודות אחרי הקידוד:', list(df_encoded.columns))

עמודות לפני הקידוד: ['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh', 'exng', 'ca', 'target']

עמודות אחרי הקידוד: ['age', 'sex', 'trtbps', 'chol', 'fbs', 'thalachh', 'exng', 'ca', 'target', 'cp_1', 'cp_2', 'cp_3', 'restecg_1', 'restecg_2']


In [8]:
# הצגת הנתונים אחרי הקידוד
df_encoded.head()

,age,sex,trtbps,chol,fbs,thalachh,exng,ca,target,cp_1,cp_2,cp_3,restecg_1,restecg_2
0,62,1,130,231,0,146,0,3,1,0,1,0,1,0
1,52,1,108,233,1,147,0,3,1,0,0,0,1,0
2,53,1,130,246,1,173,0,3,1,0,1,0,0,0
3,67,1,160,286,0,108,1,3,0,0,0,0,0,0
4,65,0,150,225,0,114,0,3,0,0,0,0,0,0


### שלב 4: הסבר התוצאה

**cp** (4 קטגוריות) הפך ל-3 עמודות:
- `cp_1` = 1 אם אנגינה טיפוסית
- `cp_2` = 1 אם אנגינה לא טיפוסית  
- `cp_3` = 1 אם כאב לא לבבי
- (כשכולם 0 = ללא תסמינים)

**restecg** (3 קטגוריות) הפך ל-2 עמודות:
- `restecg_1` = 1 אם חריגות ST-T
- `restecg_2` = 1 אם הגדלת חדר שמאל
- (כשכולם 0 = תקין)

In [9]:
# סיכום: מימדי הנתונים לפני ואחרי
print(f'לפני הקידוד: {df.shape[1]} עמודות')
print(f'אחרי הקידוד: {df_encoded.shape[1]} עמודות')
print(f'\nהוספנו {df_encoded.shape[1] - df.shape[1]} עמודות חדשות (והסרנו 2 מקוריות)')

לפני הקידוד: 11 עמודות
אחרי הקידוד: 14 עמודות

הוספנו 3 עמודות חדשות (והסרנו 2 מקוריות)


## בניית מודל רגרסיה לוגיסטית

### שלב 5: הפרדת משתנים בלתי תלויים (X) ומשתנה תלוי (y)

In [10]:
# הפרדת X (משתנים מסבירים) ו-y (משתנה היעד)
X = df_encoded.drop('target', axis=1)
y = df_encoded['target']

print('מימדי X:', X.shape)
print('מימדי y:', y.shape)

מימדי X: (298, 13)
מימדי y: (298,)


### שלב 6: פיצול לסט אימון וסט בדיקה (80% / 20%)

In [11]:
from sklearn.model_selection import train_test_split

# פיצול הנתונים: 80% אימון, 20% בדיקה
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('גודל סט אימון:', X_train.shape[0])
print('גודל סט בדיקה:', X_test.shape[0])

גודל סט אימון: 238
גודל סט בדיקה: 60


### שלב 7: בניית ואימון המודל

In [12]:
from sklearn.linear_model import LogisticRegression

# יצירת המודל
model = LogisticRegression(max_iter=1000)

# אימון המודל על סט האימון
model.fit(X_train, y_train)

print('המודל אומן בהצלחה!')

המודל אומן בהצלחה!


c:\Users\miriam\anaconda3\envs\llmenv2\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### שלב 8: חיזוי והערכת המודל

In [13]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# חיזוי על סט הבדיקה
y_pred = model.predict(X_test)

# חישוב דיוק המודל
accuracy = accuracy_score(y_test, y_pred)
print(f'דיוק המודל (Accuracy): {accuracy:.2%}')

דיוק המודל (Accuracy): 90.00%


In [14]:
# מטריצת הבלבול (Confusion Matrix)
print('מטריצת בלבול:')
print(confusion_matrix(y_test, y_pred))
print('\n' + '='*50 + '\n')

# דוח סיווג מפורט
print('דוח סיווג:')
print(classification_report(y_test, y_pred, target_names=['סיכון נמוך (0)', 'סיכון גבוה (1)']))

מטריצת בלבול:
[[24  4]
 [ 2 30]]


דוח סיווג:
                precision    recall  f1-score   support

סיכון נמוך (0)       0.92      0.86      0.89        28
סיכון גבוה (1)       0.88      0.94      0.91        32

      accuracy                           0.90        60
     macro avg       0.90      0.90      0.90        60
  weighted avg       0.90      0.90      0.90        60



### שלב 9: חישוב המטריקות - Accuracy, Precision, Recall, F1

In [15]:
from sklearn.metrics import precision_score, recall_score, f1_score

# חישוב המטריקות עבור סיכון גבוה (1)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('=== מטריקות המודל ===\n')
print(f'Accuracy:  {accuracy:.2%}  - מתוך כל התחזיות, כמה היו נכונות?')
print(f'Precision: {precision:.2%}  - מתוך מי שחזינו כחולה, כמה באמת חולים?')
print(f'Recall:    {recall:.2%}  - מתוך כל החולים האמיתיים, כמה זיהינו?')
print(f'F1-Score:  {f1:.2%}  - ממוצע הרמוני של Precision ו-Recall')

=== מטריקות המודל ===

Accuracy:  90.00%  - מתוך כל התחזיות, כמה היו נכונות?
Precision: 88.24%  - מתוך מי שחזינו כחולה, כמה באמת חולים?
Recall:    93.75%  - מתוך כל החולים האמיתיים, כמה זיהינו?
F1-Score:  90.91%  - ממוצע הרמוני של Precision ו-Recall


### שלב 10: איזו מטריקה חשובה יותר בהקשר הרפואי?

**בהקשר רפואי, Recall היא המטריקה החשובה ביותר!**

#### למה?

| סוג שגיאה | משמעות | השלכות |
|-----------|--------|--------|
| **False Negative** (פספוס) | חולה אמיתי שלא זוהה | המטופל לא יקבל טיפול → סכנת חיים! |
| **False Positive** (אזעקת שווא) | בריא שזוהה כחולה | בדיקות נוספות מיותרות → עלות כספית |

#### הסבר:
- **Recall גבוה** = פחות False Negatives = פחות חולים שפספסנו
- בתחום הרפואי, עדיף לשלוח אדם בריא לבדיקות נוספות מאשר לפספס חולה אמיתי
- פספוס התקף לב יכול להיות קטלני, בעוד אזעקת שווא רק תגרום לבדיקות נוספות

**במודל שלנו: Recall = 94%** - זיהינו 30 מתוך 32 חולים אמיתיים (פספסנו רק 2)

In [ ]:
%pip install streamlit plotly
